<a href="https://colab.research.google.com/github/Dasha-nz/nz/blob/main/%D0%9A.%D0%9B./%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
! pip install nltk transformers torch numpy

In [34]:
def explain_ud_tags():

    tagged_sentence = [
        ('Кот', 'NOUN'),
        ('спит', 'VERB'),
        ('.', 'PUNCT')
    ]

    explanations = {
        'NOUN': "Существительное, обозначающее предмет или явление.",
        'VERB': "Глагол, обозначающий действие или состояние.",
        'PUNCT': "Знаки препинания, используемые для разделения предложений."
    }

    for word, tag in tagged_sentence:
        print(f"{word} ({tag}): {explanations.get(tag, 'Неизвестный тег')}")


if __name__ == "__main__":
    print("Объяснение UD-тегов:")
    explain_ud_tags()


Объяснение UD-тегов:
Кот (NOUN): Существительное, обозначающее предмет или явление.
спит (VERB): Глагол, обозначающий действие или состояние.
. (PUNCT): Знаки препинания, используемые для разделения предложений.


In [35]:
def split_compound_sentence(sentence):
   # Функция для разбиения сложносочиненного предложения на простые.
   # :param sentence: Сложносочиненное предложение
   # :return: Список простых предложений

    parts = []
    conjunctions = ['и', 'а', 'но', 'или']

    if ',' in sentence:
        parts = [part.strip() for part in sentence.split(',')]
    else:

        for conjunction in conjunctions:
            if conjunction in sentence:
                parts = [part.strip() for part in sentence.split(conjunction)]
                break

    return parts

if __name__ == "__main__":
    complex_sentence = "Кот спит, а собака лает."
    simple_sentences = split_compound_sentence(complex_sentence)
    print("Разбиение сложносочиненного предложения:")
    for i, simple in enumerate(simple_sentences, start=1):
        print(f"Предложение {i}: {simple}")


Разбиение сложносочиненного предложения:
Предложение 1: Кот спит
Предложение 2: а собака лает.


In [36]:
class TreeNode:
    def __init__(self, token):
        self.token = token
        self.parent = None

def find_lowest_common_ancestor(node1, node2):


    ancestors1 = set()


    while node1:
        ancestors1.add(node1)
        node1 = node1.parent

        if node2 in ancestors1:
            return node2
        node2 = node2.parent

    return None

if __name__ == "__main__":

    root = TreeNode("root")
    child1 = TreeNode("child1")
    child2 = TreeNode("child2")
    grandchild1 = TreeNode("grandchild1")
    grandchild2 = TreeNode("grandchild2")


    child1.parent = root
    child2.parent = root
    grandchild1.parent = child1
    grandchild2.parent = child2


    lca = find_lowest_common_ancestor(grandchild1, grandchild2)

    if lca:
        print(f"Наименьший общий предок: {lca.token}")
    else:
        print("Наименьший общий предок не найден.")


Наименьший общий предок: root


In [37]:
import numpy as np
from nltk.tree import Tree
from nltk import edit_distance
from transformers import BertTokenizer, BertModel
import torch

def compute_edit_distance(tree1, tree2):

    str_tree1 = tree1.pformat()
    str_tree2 = tree2.pformat()


    return edit_distance(str_tree1, str_tree2)

def compute_bert_embedding(sentence):

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    inputs = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].numpy()
    return embedding

def cosine_similarity(vec1, vec2):

    vec1 = vec1.flatten()
    vec2 = vec2.flatten()
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

if __name__ == "__main__":
  sentences_pairs = [
    ("Кошка сидит на коврике.", "Кошка на коврике."),
    ("Собаки — отличные питомцы.", "Кошки — замечательные животные."),
    ("Я люблю конфетки.", "Мне нравится кушать конфетки.")
    ]


  for sent1, sent2 in sentences_pairs:

        embedding1 = compute_bert_embedding(sent1)
        embedding2 = compute_bert_embedding(sent2)


        cos_sim = cosine_similarity(embedding1, embedding2)

        tree1 = Tree.fromstring('(S (NP Кошка) (VP сидит (PP на (NP коврике))))')
        tree2 = Tree.fromstring('(S (NP Кошка) (VP есть (PP на (NP коврике))))')

        edit_dist = compute_edit_distance(tree1, tree2)

        print(f"Сравнение предложений: '{sent1}' и '{sent2}'")
        print(f"Косинусное сходство: {cos_sim:.4f}")
        print(f"Расстояние редактирования: {edit_dist}\n")


Сравнение предложений: 'Кошка сидит на коврике.' и 'Кошка на коврике.'
Косинусное сходство: 0.9868
Расстояние редактирования: 5

Сравнение предложений: 'Собаки — отличные питомцы.' и 'Кошки — замечательные животные.'
Косинусное сходство: 0.9853
Расстояние редактирования: 5

Сравнение предложений: 'Я люблю конфетки.' и 'Мне нравится кушать конфетки.'
Косинусное сходство: 0.9744
Расстояние редактирования: 5

